In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
from sklearn.model_selection import *
from sklearn.utils import shuffle 
# 导入数据
data = pd.read_csv('data/waimai_10k.csv')
data = shuffle(data)#将数据打乱
data.head()

,label,review
9323,0,根本就没送，就完成了？我想问你把饭送哪去了？也不给我个电话？逗
9125,0,恶意点确认，菜未送到，提前点击确认。
2391,1,"天气不好,非常感谢,还是热的,好评！"
11135,0,较之前还是吃的时候腊肉量偏少，不知是不是商家减量还是偶然的情况
3216,1,太实惠了，吃到撑啊！


In [2]:
# 去除英文与数字，符号,采用replace方法
data.review = data.review.str.replace('[0-9a-zA-Z]','')
data.head()

,label,review
9323,0,根本就没送，就完成了？我想问你把饭送哪去了？也不给我个电话？逗
9125,0,恶意点确认，菜未送到，提前点击确认。
2391,1,"天气不好,非常感谢,还是热的,好评！"
11135,0,较之前还是吃的时候腊肉量偏少，不知是不是商家减量还是偶然的情况
3216,1,太实惠了，吃到撑啊！


In [3]:
#文本分词
import jieba

with open("data/stop_words",encoding='utf-8') as words:
    stop_word = [i.strip() for i in words.readlines()]

    
def word_cut(word_data):
    w = [i for i in jieba.cut(word_data) if i not in stop_word]
    result = ' '.join(w)
    return result

ww = data.review.apply(word_cut)
ww[:5]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.717 seconds.
Prefix dict has been built successfully.


9323             没 送 想 问 饭 送 电话 逗
9125      恶意 点 确认 菜 未 送到 提前 点击 确认
2391              天气 不好 非常感谢 热 好评
11135    吃 腊肉 量 偏少 不知 商家 减量 偶然 情况
3216                     太 实惠 吃 撑
Name: review, dtype: object

In [4]:
# 向量化
cv=TfidfVectorizer(binary=False,decode_error='ignore',stop_words='english')
word_tf=cv.fit_transform(ww).toarray()#传入句子组成的list
word_tf.shape

(11987, 9037)

In [5]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=5)
# word_tf = pca.fit_transform(word_tf)

In [6]:
x_train,x_test,y_train,y_test = train_test_split(word_tf,data.label,test_size=0.2,random_state=0)


In [7]:
(len(x_train),len(y_train))

(9589, 9589)

In [8]:
svm_model = svm.LinearSVC()
svm_model.fit(x_train,y_train)
y_predict = svm_model.predict(x_test)
y_predict

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [9]:
print('SVM模型的准确率为：\n',accuracy_score(y_test, y_predict))
print('SVM模型的评估报告：\n',classification_report(y_test, y_predict))

SVM模型的准确率为：
 0.8386155129274395
SVM模型的评估报告：
              precision    recall  f1-score   support

          0       0.86      0.90      0.88      1582
          1       0.79      0.71      0.75       816

avg / total       0.84      0.84      0.84      2398

